# OpenAI Supervised Fine-Tuning

This recipe allows TensorZero users to fine-tune OpenAI models using their own data.
Since TensorZero automatically logs all inferences and feedback, it is straightforward to fine-tune a model using your own data and any prompt you want.


To get started:

- Set the `CLICKHOUSE_URL` environment variable. For example: `CLICKHOUSE_URL="http://localhost:8123/tensorzero"`
- Set the `OPENAI_API_KEY` environment variable.
- Update the following parameters:


In [1]:
CONFIG_PATH = "../../../examples/haiku-hidden-preferences/config/tensorzero.toml"

FUNCTION_NAME = "write_haiku"

METRIC_NAME = "haiku_score"

# The name of the variant to use to grab the templates used for fine-tuning
TEMPLATE_VARIANT_NAME = "initial_prompt_gpt4o_mini"

# If the metric is a float metric, you can set the threshold to filter the data
FLOAT_METRIC_THRESHOLD = 0.5

# Fraction of the data to use for validation
VAL_FRACTION = 0.2

# Maximum number of samples to use for fine-tuning
MAX_SAMPLES = 100_000

# The name of the model to fine-tune (supported models: https://platform.openai.com/docs/guides/fine-tuning)
MODEL_NAME = "gpt-4o-mini-2024-07-18"

In [2]:
import json
import os
import tempfile
import time
from pathlib import Path
from pprint import pprint
from typing import Any, Dict, List

import numpy as np
import openai
import pandas as pd
import toml
from clickhouse_connect import get_client
from IPython.display import clear_output
from minijinja import Environment

Load the TensorZero configuration file.


In [3]:
config_path = Path(CONFIG_PATH)

assert config_path.exists(), f"{CONFIG_PATH} does not exist"
assert config_path.is_file(), f"{CONFIG_PATH} is not a file"

with config_path.open("r") as f:
    config = toml.load(f)

Retrieve the metric configuration.


In [4]:
assert "metrics" in config, "No `[metrics]` section found in config"
assert (
    METRIC_NAME in config["metrics"]
), f"No metric named `{METRIC_NAME}` found in config"

metric = config["metrics"][METRIC_NAME]

metric

{'type': 'boolean', 'level': 'inference', 'optimize': 'max'}

Retrieve the configuration for the variant with the templates we'll use for fine-tuning.


In [5]:
assert "functions" in config, "No `[functions]` section found in config"
assert (
    FUNCTION_NAME in config["functions"]
), f"No function named `{FUNCTION_NAME}` found in config"
assert (
    "variants" in config["functions"][FUNCTION_NAME]
), f"No variants section found for function `{FUNCTION_NAME}`"
assert (
    TEMPLATE_VARIANT_NAME in config["functions"][FUNCTION_NAME]["variants"]
), f"No variant named `{TEMPLATE_VARIANT_NAME}` found in function `{FUNCTION_NAME}`"

function_type = config["functions"][FUNCTION_NAME]["type"]
variant = config["functions"][FUNCTION_NAME]["variants"][TEMPLATE_VARIANT_NAME]

variant

{'type': 'chat_completion',
 'weight': 0,
 'model': 'gpt-4o-mini-2024-07-18',
 'system_template': 'functions/write_haiku/initial_prompt/system_template.minijinja',
 'user_template': 'functions/write_haiku/initial_prompt/user_template.minijinja'}

Retrieve the system, user, and assistant templates in the variant (if any), and initialize a minijinja environment with them.


In [6]:
templates = {}

if "assistant_template" in variant:
    assistant_template_path = config_path.parent / variant["assistant_template"]
    with assistant_template_path.open("r") as f:
        templates["assistant"] = f.read()

if "system_template" in variant:
    system_template_path = config_path.parent / variant["system_template"]
    with system_template_path.open("r") as f:
        templates["system"] = f.read()

if "user_template" in variant:
    user_template_path = config_path.parent / variant["user_template"]
    with user_template_path.open("r") as f:
        templates["user"] = f.read()

env = Environment(templates=templates)

Initialize the ClickHouse client.


In [7]:
assert "CLICKHOUSE_URL" in os.environ, "CLICKHOUSE_URL environment variable not set"

clickhouse_client = get_client(dsn=os.environ["CLICKHOUSE_URL"])

Determine the ClickHouse table name for the metric.


In [8]:
feedback_table_name = {
    "float": "FloatMetricFeedback",
    "boolean": "BooleanMetricFeedback",
}.get(metric["type"])

if feedback_table_name is None:
    raise ValueError(f"Unsupported metric type: {metric['type']}")

Determine the correct join key to use for the metric on the inference table.


In [9]:
inference_join_key = {
    "episode": "episode_id",
    "inference": "id",
}.get(metric["level"])

if inference_join_key is None:
    raise ValueError(f"Unsupported metric level: {metric['level']}")

Query the inferences and feedback from ClickHouse.

If the metric is a float metric, we need to filter the data based on the threshold.


In [10]:
assert "optimize" in metric, "Metric is missing the `optimize` field"

threshold = FLOAT_METRIC_THRESHOLD if metric["type"] == "float" else 0.5
comparison_operator = ">=" if metric["optimize"] == "max" else "<="

query = f"""
SELECT 
    i.variant_name, 
    i.input, 
    i.output, 
    f.value,
    i.episode_id
FROM 
    Inference i
JOIN 
    (SELECT
        target_id,
        value,
        ROW_NUMBER() OVER (PARTITION BY target_id ORDER BY timestamp DESC) as rn
    FROM 
        {feedback_table_name}
    WHERE
        metric_name = %(metric_name)s
        AND value {comparison_operator} %(threshold)s
    ) f ON i.{inference_join_key} = f.target_id and f.rn = 1
WHERE 
    i.function_name = %(function_name)s
LIMIT %(max_samples)s
"""

params = {
    "function_name": FUNCTION_NAME,
    "metric_name": METRIC_NAME,
    "comparison_operator": comparison_operator,
    "threshold": threshold,
    "max_samples": MAX_SAMPLES,
}

df = clickhouse_client.query_df(query, params)

df.head()

,variant_name,input,output,value,episode_id
0,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""\""Capacity\"" evokes th...",True,0191d773-3de0-7652-827c-7f2114fc584b
1,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Preoccupation can ofte...",True,0191d773-ae66-7833-8763-7d15cc6853ae
2,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Neurons spark and fire...",True,0191d773-4c08-7e20-87d2-27f7d5f6a5c2
3,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""A sunbonnet is a charm...",True,0191d773-17f4-7231-8838-8c10b31a8a04
4,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Length can be perceive...",True,0191d773-041f-7383-891f-cd13727fec7b


Render the inputs using the templates.


In [11]:
def render_message(content: List[Dict[str, Any]], role: str) -> str:
    assert role in ["user", "assistant"], f"Invalid role: {role}"

    if len(content) != 1:
        raise ValueError(f"Message must have exactly one content block: {content}")

    if content[0]["type"] != "text":
        raise ValueError(f"Content block must be of type text: {content}")

    content = content[0]["value"]

    if isinstance(content, str):
        return content
    else:
        return env.render_template(role, **content)


def sample_to_openai_messages(sample) -> List[Dict[str, str]]:
    function_input = json.loads(sample["input"])

    rendered_messages = []

    # Add the system message to the rendered messages
    # If there is data passed in or a system template there must be a system message
    system = function_input.get("system", {})
    if len(system) > 0 or system_template_path:
        if system_template_path:
            system_message = env.render_template("system", **system)
            rendered_messages.append({"role": "system", "content": system_message})
        else:
            rendered_messages.append({"role": "system", "content": system})

    # Add the input messages to the rendered messages
    for message in function_input["messages"]:
        rendered_message = render_message(message["content"], message["role"])
        rendered_messages.append({"role": message["role"], "content": rendered_message})

    # Add the output to the messages
    output = json.loads(sample["output"])

    if function_type == "chat":
        if len(output) != 1:
            raise ValueError(f"Output {output} must have exactly one content block.")

        if output[0]["type"] != "text":
            raise ValueError(f"Output {output} must be a text block.")

        rendered_messages.append({"role": "assistant", "content": output[0]["text"]})
    elif function_type == "json":
        rendered_messages.append({"role": "assistant", "content": output["raw"]})
    else:
        raise ValueError(f"Unsupported function type: {function_type}")

    return {"messages": rendered_messages}


df["openai_messages"] = df.apply(sample_to_openai_messages, axis=1)

df.head()

,variant_name,input,output,value,episode_id,openai_messages
0,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""\""Capacity\"" evokes th...",True,0191d773-3de0-7652-827c-7f2114fc584b,"{'messages': [{'role': 'system', 'content': 'Y..."
1,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Preoccupation can ofte...",True,0191d773-ae66-7833-8763-7d15cc6853ae,"{'messages': [{'role': 'system', 'content': 'Y..."
2,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Neurons spark and fire...",True,0191d773-4c08-7e20-87d2-27f7d5f6a5c2,"{'messages': [{'role': 'system', 'content': 'Y..."
3,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""A sunbonnet is a charm...",True,0191d773-17f4-7231-8838-8c10b31a8a04,"{'messages': [{'role': 'system', 'content': 'Y..."
4,initial_prompt_gpt4o_mini,"{""messages"":[{""role"":""user"",""content"":[{""type""...","[{""type"":""text"",""text"":""Length can be perceive...",True,0191d773-041f-7383-891f-cd13727fec7b,"{'messages': [{'role': 'system', 'content': 'Y..."


Split the data into training and validation sets for fine-tuning.


In [12]:
# Get unique episode_ids
unique_episode_ids = df["episode_id"].unique()

# Shuffle the unique episode_ids
np.random.seed(42)
np.random.shuffle(unique_episode_ids)

# Calculate the split index for episode_ids
split_index = int(len(unique_episode_ids) * (1 - VAL_FRACTION))

# Split the episode_ids into training and validation sets
train_episode_ids = unique_episode_ids[:split_index]
val_episode_ids = unique_episode_ids[split_index:]

# Create training and validation DataFrames based on episode_ids
train_df = df[df["episode_id"].isin(train_episode_ids)]
val_df = df[df["episode_id"].isin(val_episode_ids)]

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Actual validation fraction: {len(val_df) / len(df):.2f}")

Training set size: 40
Validation set size: 11
Actual validation fraction: 0.22


Upload the training and validation datasets to OpenAI.


In [13]:
def upload_dataset_to_openai(df: pd.DataFrame, openai_client: openai.OpenAI) -> str:
    with tempfile.NamedTemporaryFile(mode="w", suffix=".jsonl", delete=False) as f:
        # Write the openai_messages to the temporary file
        for item in df["openai_messages"]:
            json.dump(item, f)
            f.write("\n")
        f.flush()

        # Upload the file to OpenAI
        with open(f.name, "rb") as file:
            file_object = openai_client.files.create(file=file, purpose="fine-tune")

        return file_object.id


openai_client = openai.OpenAI()

train_file_object_id = upload_dataset_to_openai(train_df, openai_client)
val_file_object_id = upload_dataset_to_openai(val_df, openai_client)

Launch the fine-tuning job.


In [14]:
fine_tuning_job = openai_client.fine_tuning.jobs.create(
    training_file=train_file_object_id,
    validation_file=val_file_object_id,
    model=MODEL_NAME,
)

Wait for the fine-tuning job to complete.

This cell will take a while to run.


In [ ]:
while True:
    clear_output(wait=True)

    try:
        job_status = openai_client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
        pprint(job_status.to_dict())
        if job_status.status in ("succeeded", "failed", "cancelled"):
            break
    except Exception as e:
        print(f"Error: {e}")

    time.sleep(10)

{'created_at': 1725896773,
 'error': {},
 'estimated_finish': 1725897095,
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'batch_size': 1,
                     'learning_rate_multiplier': 1.8,
                     'n_epochs': 3},
 'id': 'ftjob-86aMde6jQcflp3Cse1rWaaNo',
 'integrations': [],
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'fine_tuning.job',
 'organization_id': 'org-fewHWgmYjDeYGco5co60C7fh',
 'result_files': [],
 'seed': 1628356953,
 'status': 'running',
 'trained_tokens': None,
 'training_file': 'file-1WQxUMMsYXqAODcTD4BHJ7u6',
 'user_provided_suffix': None,
 'validation_file': 'file-kJZdrKjqTSZGWEywklVtDaaV'}


Once the fine-tuning job is complete, you can add the fine-tuned model to your config file.


In [ ]:
fine_tuned_model = job_status.fine_tuned_model
model_config = {
    "models": {
        fine_tuned_model: {
            "routing": ["openai"],
            "providers": {"openai": {"type": "openai", "model_name": fine_tuned_model}},
        }
    }
}

print(toml.dumps(model_config))

Finally, add a new variant to your function to use the fine-tuned model.


In [ ]:
variant_config = {
    "type": "chat_completion",
    "weight": 0,
    "model": fine_tuned_model,
}

system_template = variant.get("system_template")
if system_template:
    variant_config["system_template"] = system_template

user_template = variant.get("user_template")
if user_template:
    variant_config["user_template"] = user_template

assistant_template = variant.get("assistant_template")
if assistant_template:
    variant_config["assistant_template"] = assistant_template

full_variant_config = {
    "functions": {FUNCTION_NAME: {"variants": {fine_tuned_model: variant_config}}}
}

print(toml.dumps(full_variant_config))

You're all set!

You can change the weight to enable a gradual rollout of the new model.

You might also add other parameters (e.g. `max_tokens`, `temperature`) to the variant section in the config file.
